In [ ]:


#import requried library
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Download the data set and unzip it
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Set up the cat and dog training and validation directories
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat and dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat and dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Verify the first 10 frams name of cat and dog list 
train_cat_fnames = os.listdir(train_cats_dir)
train_cat_fnames.sort()
print(train_cat_fnames[:10])
train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

# Print the total number of cat and dog images in training and validation directories
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

# Let's look at some of the images

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
nrows = 4
ncols = 4
pic_index=0

# Set up matplotlib fig. and size it to fit 4X4 pics
fig=plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index+=8
next_cat_pix=[os.path.join(train_cats_dir, fname)
    for fname in train_cat_fnames[pic_index-8:pic_index]
    ]
next_dog_pix=[os.path.join(train_dogs_dir, fname)
    for fname in train_dog_fnames[pic_index-8:pic_index]
    ]
  
for i, img_path in enumerate(next_cat_pix+next_dog_pix):
# Set up subplot; subplot indices start at 1
  sp= plt.subplot(nrows, ncols, i+1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img= mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


In [ ]:
# Building a model

from tensorflow.keras import models, layers

model = models.Sequential([
	# Note that the input shape is the desired size of the image 150x150 with 3 bytes color
	layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
	layers.MaxPooling2D(2,2),
	layers.Conv2D(32, (3,3), activation='relu'),
	layers.MaxPooling2D(2,2),
	layers.Conv2D(64, (3,3), activation='relu'),
	layers.MaxPooling2D(2,2),
	# Flatten the results to feed into a DNN
	layers.Flatten(),
	# 512 neuron hidden layer
	layers.Dense(512, activation='relu'),
	# Only 1 ouput neuron. It will contain a value from 0-1, 0 for cat class and 1 for dog class
	layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
# Compile model

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
			loss='binary_crossentropy',
			metrics=['acc'])

In [ ]:
# Data Preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

# ----------
# Flow training images in batches of 20 using train_datagen generator
# ----------
train_generator = train_datagen.flow_from_directory(train_dir,
											batch_size=20,
											class_mode='binary',
											target_size=(150,150))

# ----------
# Flow validation images in batches of 20 using test_datagen generator
# ----------
validation_generator = test_datagen.flow_from_directory(validation_dir,
											batch_size=20,
											class_mode='binary',
											target_size=(150,150))

In [ ]:
# Training

history = model.fit_generator(train_generator,
						validation_data=validation_generator,
						steps_per_epoch=100,
						epochs=15,
						validation_steps=50,
						verbose=2)

In [ ]:
# Running the model

import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
	# Predicting images
	path = '/content/' + fn
	img = image.load_img(path, target_size=(150, 150))

	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	images = np.vstack([x])

	classes = model.predict(images, batch_size=10)

	print(classes[0])

	if classes[0] > 0:
		print(fn + " is a dog")
	else:
		print(fn + " is a cat")

In [ ]:
# Visualizing Intermediate Representations

import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(
    inputs=model.input, outputs=successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]

img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
	if len(feature_map.shape) == 4:
		# -------------------------------------------
		# Just do this for the conv / maxpool layers, not the fully-connected layers
		# -------------------------------------------
		# number of features in the feature map
		n_features = feature_map.shape[-1]
		# feature map shape (1, size, size, n_features)
		size = feature_map.shape[1]

		# We will tile our images in this matrix
		display_grid = np.zeros((size, size * n_features))

		# -------------------------------------------------
		# Postprocess the feature to be visually palatable
		# -------------------------------------------------
		for i in range(n_features):
			x = feature_map[0, :, :, i]
			x -= x.mean()
			x /= x.std()
			x *= 64
			x += 128
			x = np.clip(x, 0, 255).astype('uint8')
			# Tile each filter into a horizontal grid
			display_grid[:, i * size: (i + 1) * size] = x
		# -----------------
		# Display the grid
		# -----------------
		scale = 20. / n_features
		plt.figure(figsize=(scale * n_features, scale))
		plt.title(layer_name)
		plt.grid(False)
		plt.imshow(display_grid, aspect='auto', cmap='viridis')


In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')
plt.figure()